Ordering system for a restaurant

https://docs.google.com/document/d/1uDn7Vxkv7nlsB-kON8pACLYuKQm2h9K56DANDGp-idQ/edit




In [15]:
# construct a dictionary of purchaseable items and the associated price and preperation time
item_menu = {
    "sandwich" : (10, 10), 
    "salad" : (8, 8),
    "soup" : (6, 15),
    "coffee" : (5, 5),
    "tea" : (5, 5)}

In [46]:
def calculate_total_price_with_tax(subtotal, tax_rate):
    """calculates the grand total including tax"""
    total_price = 0

    ## basic validity check
    if len(subtotal) > 0:
        for item in subtotal:
            total_price += subtotal[item][1]
            
    # calculate tax (in Irvine, Ca 7.25%)
    tax = total_price * 0.0725
    # calculate grand total price
    grand_total = total_price + tax
    return grand_total, tax


In [50]:
def calculate_total_time(subtotal):
    total_time = 0
    ## basic validity check
    if len(subtotal) > 0:
        for item in subtotal:
            total_time += subtotal[item][2]

    return total_time

In [48]:
import datetime

def print_receipt(customer_name, subtotal):
    # assumed tax rate
    tax_rate = 0.0725

    # get the additional dis
    date = datetime.date.today()
    
    # calculate the total price (grand total including tax)
    grand_total, tax = calculate_total_price_with_tax(subtotal, tax_rate)
    
    # calculate total time in minutes
    total_time = calculate_total_time(subtotal)
    
    # print the receipt in pretty format
    ## create receipt header
    output_str = "*************************************************\n"
    output_str += f"{customer_name}, thanks for your order\n\n"
    output_str += "Items     qty       Price\n"
    ## create item list
    for key, vals in subtotal.items():
        item_name = key
        qty = vals[0]
        price = vals[1]
        output_str += f"{item_name:<10}{qty:<10}{price:.2f}\n"
    ## add receipt tax and totals
    output_str += "\n"
    output_str += f"Tax       ${tax:.2f}\n"
    output_str += f"Total     ${grand_total:.2f}\n"
    ## add receipt footer
    output_str += f"{date},  Your order will be ready in {total_time} minutes\n"
    output_str += "***************************************************"
    
    print(output_str)

In [18]:
def get_tea_subtotal(cart):
    """return the total quantity, price, and preperation time
    input is everything required to include discounts
    future: even though there is not discount today, we keep
    a dedicated function to calculate tea grand total so it
    is easy to add in the future if we decide to
    """
    nb_tea = cart["tea"]
    price_per_tea = item_menu["tea"][0]
    min_per_tea = item_menu["tea"][1]
    total_min = nb_tea * min_per_tea
    total_price = nb_tea * price_per_tea
    return nb_tea, total_price, total_min

In [19]:
def get_coffee_subtotal(cart):
    """return the total quantity, price, and preperation time
    input is everything required to include discounts
    future: even though there is not discount today, we keep
    a dedicated function to calculate coffee grand total so it
    is easy to add in the future if we decide to
    """
    nb_coffee = cart["coffee"]
    price_per_coffee = item_menu["coffee"][0]
    min_per_coffee = item_menu["coffee"][1]
    total_time = nb_coffee * min_per_coffee
    total_price = nb_coffee * price_per_coffee
    return nb_coffee, total_price, total_time

In [20]:
def get_soup_subtotal(cart):
    """return the total quantity, price, and preperation time
    input is everything required to include discounts
    """
    soup_total = None
    nb_soup = cart["soup"]
    nb_sandwich = cart["sandwich"] if "sandwich" in cart else 0
    nb_salad = cart["salad"] if "salad" in cart else 0
    price_per_soup = item_menu["soup"][0]
    min_per_salad = item_menu["soup"][1]
    total_time = nb_soup * min_per_salad
    # if ordered with a sandwich AND salad, apply the 20% discount
    if nb_sandwich > 0 or nb_salad > 0:
        # determine how many soups get discounted. Only discount a soup if there is matching
        # soup AND sandwich
        ## first, determine if we have less sandwiches or salads. This will determine the theorhetical
        ## maximum match to how many soups we can discount
        nb_max_possible_discount = nb_sandwich if nb_sandwich < nb_salad else nb_salad
        if nb_max_possible_discount < nb_soup:
            # apply discount to as many salads as soups, then add the full price of the difference
            total_price = nb_max_possible_discount * price_per_soup * 0.80
            total_price += (nb_soup - nb_max_possible_discount) * price_per_soup
            soup_total = nb_soup, total_price, total_time
        else:
            # apply discount to all soups
            total_price = nb_soup * price_per_soup * 0.80
            soup_total = nb_soup, total_price, total_time
    else:
        total_price = nb_soup * price_per_soup
        soup_total = nb_soup, total_price, total_time
    
    return soup_total

In [21]:
def get_salad_subtotal(cart):
    """return the total quantity, price, and preperation time
    input is everything required to include discounts
    """
    salad_total = None
    nb_salad = cart["salad"]
    nb_soup = cart["soup"] if "soup" in cart else 0
    price_per_salad = item_menu["salad"][0]
    min_per_salad = item_menu["salad"][1]
    total_time = nb_salad * min_per_salad
    # if ordered with a soup, the 10% discount
    if nb_soup> 0:
        # determine how many salads get discounted. Only discount a salad if there is matching soup
        if nb_soup < nb_salad:
            # apply discount to as many salads as soups, then add the full price of the difference
            total_price = nb_soup * price_per_salad * 0.90
            total_price += (nb_salad - nb_soup) * price_per_salad
            salad_total = nb_salad, total_price, total_time
        else:
            # apply discount to all salads
            total_price = nb_salad * price_per_salad * 0.90
            salad_total = nb_salad, total_price, total_time
    else:
        total_price = nb_salad * price_per_salad
        salad_total = nb_salad, total_price, total_time
    
    return salad_total

In [22]:
def get_sandwich_subtotal(cart):
    """return the total quantity, price, and preperation time
    input is everything required to include discounts
    """
    sandwich_total = None
    nb_sandwich = cart["sandwich"]
    price_per_sandwich = item_menu["sandwich"][0]
    min_per_sandwich = item_menu["sandwich"][1]
    total_time = nb_sandwich * min_per_sandwich
    # if 5 or more sandwiches are ordered, appy a 10% discount
    if nb_sandwich >= 5:
        total_price = nb_sandwich * price_per_sandwich * 0.90
        sandwich_total = nb_sandwich, total_price, total_time
    else:
        total_price = nb_sandwich * price_per_sandwich
        sandwich_total = nb_sandwich, total_price, total_time
        
    return sandwich_total

In [23]:
def calculate_subtotal(cart):
    order_total = {}

    # calculate sandwich total
    if "sandwich" in cart:
        order_total["sandwich"] = get_sandwich_subtotal(cart)
        
    # calculate salad total
    if "salad" in cart:
        order_total["salad"] = get_salad_subtotal(cart)
        
    # calculate salad total
    if "soup" in cart:
        order_total["soup"] = get_soup_subtotal(cart)
        
    # calculate coffee total
    if "coffee" in cart:
        order_total["coffee"] = get_coffee_subtotal(cart)
        
    # calculate tea total
    if "tea" in cart:
        order_total["tea"] = get_tea_subtotal(cart)
        
    return order_total

In [24]:
def prompt_for_item_and_quantity():  
    # get item type
    ## initial prompt for item type
    item = input("Please enter item you want to purchase:")
    ## basic validity check to make sure they enter a valid item
    while item not in item_menu:
        item = input(f"{item} is not available to purchase. Please enter item you want to purchase:")

    # get item quantity
    ## initial prompt for item quantity
    quantity = input("Please enter quantity that you want:")
    ## basic validity check
    while not quantity.isdigit():
        quantity = input(f"{quantity} is not a valid quantity. Please enter quantity that you want:")
    quantity = int(quantity)

    return item, quantity

In [25]:
def get_order_cart():
    """Gets the order cart from the user
    Launches an interactive dialog with the user to get the
    items and quantity of items they want to purchase. Returns
    the item sub total - the \'cart\'
    """
    
    # create a dictionary to capture the order (item and quantity) prior to any discounts
    order_subtotal = {}
    
    # ask for more items over a loop
    is_order_more = "y"
    # enter loop prompt
    while is_order_more == 'y':
        # if user want to order more, prompt them for which item and quantity
        item, quantity = prompt_for_item_and_quantity()
        # add to running order order
        ## check if key exists. if it does, add to the existing quantity. otherwise, create the key and add the quantitiy
        if item in order_subtotal:
            order_subtotal[item] += quantity
        else:
            order_subtotal[item] = quantity
        # prompt user if they would like to order more
        is_order_more = input("Would you like to order more? (y)es or (n)o:")
        ## perform a basic validity check for the answer
        while is_order_more.lower() != "y" and is_order_more.lower() !=  "n":
            print("Invalid response.")
            is_order_more = input("Would you like to order more? (y)es or (n)o:")
        
    return order_subtotal
    

In [26]:
def take_order():
    # prompt for name
    customer_name = input("Please give me your name: ")

    # get the cart from customer
    cart = get_order_cart()

    # compute order details after applying discounts (item, quantity, price, and time)
    subtotal = calculate_subtotal(cart)

    # print receipt
    print_receipt(customer_name, subtotal)


In [15]:
if __name__ == "__main__":
    take_order()

# Test Cases

## Test get_sandwich_subtotal()

In [ ]:
"""Sandwiches are discounted 10% after 5 quantity. This boundary tests the sandwich subtotal function"""
cart = {} # dictionary - key == item, value == qty
price = 10
time = 10
discount_adjustment = 0.90 # discount is 10% off, or 90% of the original price

# less than 5
qty = 4
cart.clear()
cart["sandwich"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_sandwich_subtotal(cart)
print(qty_actual, total_price_actual, total_time_actual)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# exactly 5
qty = 5
cart.clear()
cart["sandwich"] = qty
total_price_expected = qty * price * discount_adjustment
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_sandwich_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# greater than 5
qty = 6
cart.clear()
cart["sandwich"] = qty
total_price_expected = qty * price * discount_adjustment
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_sandwich_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

print("All tests passed")

## Test get_salad_subtotal

In [ ]:
"""Salads are discounted 10% if ordered with a soup. This tetss that salads
are only discounted up to the same number of soups ordered"""
cart = {} # dictionary - key == item, value == qty
price = 8
time = 8
discount_adjustment = 0.90 # discount is 10% off, or 90% of the original price

# 1 salad only
qty = 1
cart.clear()
cart["salad"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_salad_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 salads only
qty = 2
cart.clear()
cart["salad"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_salad_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 salads and 1 soup
qty_salad = 2
qty_soup = 1
cart.clear()
cart["salad"] = qty_salad
cart["soup"] = qty_soup
total_price_expected = price + (price * discount_adjustment) # discount one salad because we have one soup, other salad is full price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_salad_subtotal(cart)
assert qty_salad == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 salads and 2 soup
qty_salad = 2
qty_soup = 2
cart.clear()
cart["salad"] = qty_salad
cart["soup"] = qty_soup
total_price_expected = qty_salad * price * discount_adjustment # discount both salads because we have two matching soups
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_salad_subtotal(cart)
assert qty_salad == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 salads and 3 soup
qty_salad = 2
qty_soup = 3
cart.clear()
cart["salad"] = qty_salad
cart["soup"] = qty_soup
total_price_expected = qty_salad * price * discount_adjustment # discount both salads because we have two matching soups. third soup does not contribute to discount
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_salad_subtotal(cart)
assert qty_salad == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

print("All tests passed")

## Test get_soup_subtotal

In [ ]:
"""Soups are discounted 20% if ordered with BOTH sandwich and salad. This tests
that soups are only discounted up to the same number of sandwiches and salads ordered"""
cart = {} # dictionary - key == item, value == qty
price = 6
time = 15
discount_adjustment = 0.80 # discount is 20% off, or 90% of the original price

# 1 soup only
qty = 1
cart.clear()
cart["soup"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_soup_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 soups only
qty = 2
cart.clear()
cart["soup"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_soup_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 soup and 1 sandwich
qty_soup = 2
qty_sandwich = 1
cart.clear()
cart["soup"] = qty_soup
cart["sandwich"] = qty_sandwich
total_price_expected = qty * price # no discount because we have 1 sandwich but no salad
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_soup_subtotal(cart)
assert qty_soup == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 soup and 1 salad
qty_soup = 2
qty_salad = 1
cart.clear()
cart["soup"] = qty_soup
cart["salad"] = qty_salad
total_price_expected = qty_soup * price # no discount because we have 1 salad but no sandwich
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_soup_subtotal(cart)
print(qty_actual, total_price_actual, total_time_actual)
assert qty_soup == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 soup and 1 sandwich and 1 salad
qty_soup = 2
qty_sandwich = 1
qty_salad = 1
cart.clear()
cart["soup"] = qty_soup
cart["sandwich"] = qty_sandwich
cart["salad"] = qty_salad
total_price_expected = price + (price * discount_adjustment) # # discount one soup because we have one sandwich+salad pair, other soup is full price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_soup_subtotal(cart)
assert qty_soup == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 soup and 1 sandwich and 2 salad
qty_soup = 2
qty_sandwich = 1
qty_salad = 2
cart.clear()
cart["soup"] = qty_soup
cart["sandwich"] = qty_sandwich
cart["salad"] = qty_salad
total_price_expected = price + (price * discount_adjustment) # # discount one soup because we have one sandwich+salad pair, other soup is full price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_soup_subtotal(cart)
assert qty_soup == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 soup and 2 sandwich and 2 salad
qty_soup = 2
qty_sandwich = 2
qty_salad = 2
cart.clear()
cart["soup"] = qty_soup
cart["sandwich"] = qty_sandwich
cart["salad"] = qty_salad
total_price_expected = qty_soup * price * discount_adjustment # discount both salads because we have two sandwich+salad pair
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_soup_subtotal(cart)
assert qty_soup == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 soup and 3 sandwich and 2 salad
qty_soup = 2
qty_sandwich = 3
qty_salad = 2
cart.clear()
cart["soup"] = qty_soup
cart["sandwich"] = qty_sandwich
cart["salad"] = qty_salad
total_price_expected = qty_soup * price * discount_adjustment # discount both salads because we have two sandwich+salad pair
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_soup_subtotal(cart)
assert qty_soup == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 soup and 3 sandwich and 3 salad
qty_soup = 2
qty_sandwich = 3
qty_salad = 2
cart.clear()
cart["soup"] = qty_soup
cart["sandwich"] = qty_sandwich
cart["salad"] = qty_salad
total_price_expected = qty_soup * price * discount_adjustment # discount both salads because we have two sandwich+salad pair
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_soup_subtotal(cart)
assert qty_soup == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

print("All tests passed")

## Test get_coffee_subtotal

In [ ]:
"""Coffee is never discounted. This goes against my morning ritual..."""
cart = {} # dictionary - key == item, value == qty
price = 5
time = 5

# 1 coffee
qty = 1
cart.clear()
cart["coffee"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_coffee_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 coffee
qty = 2
cart.clear()
cart["coffee"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_coffee_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 10 coffee
qty = 10
cart.clear()
cart["coffee"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_coffee_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

print("All tests passed")

## Test get_tea_subtotal

In [29]:
"""Tes is never discounted"""
cart = {} # dictionary - key == item, value == qty
price = 5
time = 5

# 1 tea
qty = 1
cart.clear()
cart["tea"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_tea_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 2 tea
qty = 2
cart.clear()
cart["tea"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_tea_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

# 10 tea
qty = 10
cart.clear()
cart["tea"] = qty
total_price_expected = qty * price
total_time_expected = qty * time
qty_actual, total_price_actual, total_time_actual = get_tea_subtotal(cart)
assert qty == qty_actual
assert total_price_expected == total_price_actual
assert total_time_expected == total_time_actual

print("All tests passed")

All tests passed


## Test calculate_subtotal

In [41]:
cart["sandwich"] = 6
cart["salad"] = 3
cart["soup"] = 4
cart["coffee"] = 2
cart["tea"] = 1

subtotal = calculate_subtotal(cart)

# assert sandwich subtotal is correct
san_qty, san_price, san_time =  subtotal["sandwich"]
assert san_qty == 6
assert san_price == san_qty * 10 * 0.9 # sandwich qty * sandwich price * discount modifier - discount all since > 5 sandwiches
assert san_time == san_qty * 10

# assert salad subtotal is correct
sld_qty, sld_price, sld_time =  subtotal["salad"]
assert sld_qty == 3
assert sld_price == sld_qty * 8 * 0.9 # salad qty * salad price * discount modifier - discount all since matching soup
assert sld_time == sld_qty * 8

# assert soup subtotal is correct
sp_qty, sp_price, sp_time =  subtotal["soup"]
assert sp_qty == 4
assert sp_price == (3 * 6 * 0.8) + 6 # discount only 3 that has matching sandwich and salad. 4th item at full price
assert sp_time == sp_qty * 15

# assert coffee subtotal is correct
c_qty, c_price, c_time =  subtotal["coffee"]
assert c_qty == 2
assert c_price == 2 * 5 # full price always
assert c_time == c_qty * 5

# assert tea subtotal is correct
c_qty, c_price, c_time =  subtotal["tea"]
assert c_qty == 1
assert c_price == 5 # full price always
assert c_time == c_qty * 5

print("All tests passed")

All tests passed


## Test Receipt output and format

In [51]:
import datetime

cart["sandwich"] = 6
cart["salad"] = 3
cart["soup"] = 4
cart["coffee"] = 2
cart["tea"] = 1

subtotal = calculate_subtotal(cart)
print_receipt("Foo Bar", subtotal)

is_verify_pass = input("Does the receipt have the name=\'Foo Bar\', (y)es or (n)o") == 'y'
assert is_verify_pass

is_verify_pass = input("Does the receipt say sandwich qty=6 price=54.00, (y)es or (n)o") == 'y'
assert is_verify_pass

is_verify_pass = input("Does the receipt say salad qty=3 price=21.60, (y)es or (n)o") == 'y'
assert is_verify_pass

is_verify_pass = input("Does the receipt say soup qty=4 price=20.40, (y)es or (n)o") == 'y'
assert is_verify_pass

is_verify_pass = input("Does the receipt say coffee qty=2 price=10.00, (y)es or (n)o") == 'y'
assert is_verify_pass

is_verify_pass = input("Does the receipt say tea qty=1 price=5.00, (y)es or (n)o") == 'y'
assert is_verify_pass

is_verify_pass = input("Does the receipt say tax=$8.05, (y)es or (n)o") == 'y'
assert is_verify_pass

is_verify_pass = input("Does the receipt say total=$119.05, (y)es or (n)o") == 'y'
assert is_verify_pass

is_verify_pass = input(f"Does the receipt have the date {datetime.date.today()}, (y)es or (n)o") == 'y'
assert is_verify_pass

is_verify_pass = input(f"Does the receipt say the order will be ready in 159 minutes, (y)es or (n)o") == 'y'
assert is_verify_pass

print("All tests passed")

*************************************************
Foo Bar, thanks for your order

Items     qty       Price
sandwich  6         54.00
salad     3         21.60
soup      4         20.40
coffee    2         10.00
tea       1         5.00

Tax       $8.05
Total     $119.05
2022-07-17,  Your order will be ready in 159 minutes
***************************************************


AssertionError: 